<a href="https://colab.research.google.com/github/kota-mizu/smbc_price_forecasting/blob/main/Training_Inference_0607.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://signate.jp/competitions/1634



In [115]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [116]:
# ライブラリのインポート
import os, warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


from IPython.display import display
import lightgbm as lgb
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error, r2_score



warnings.filterwarnings("ignore")

# 日本語フォントを簡単に使う
!pip -q install japanize-matplotlib
import japanize_matplotlib

In [155]:
data_path = '/content/drive/MyDrive/signate_smbc/'
submission_path = '/content/drive/MyDrive/signate_smbc/submission/'

In [118]:
train_raw_df  = pd.read_csv(data_path + 'train.csv', index_col=0)
test_raw_df   = pd.read_csv(data_path + 'test.csv', index_col=0)
sample_submission_df = pd.read_csv(data_path + 'sample_submit.csv', header=None)  # header 無し
feature_description = pd.read_csv(data_path + 'feature_description.csv', index_col = 0)

## 前処理

In [119]:
train_raw_df["seville_temp"]- 273.15

,seville_temp
time,
2015-01-02 00:00:00+01:00,3.379
2015-01-02 01:00:00+01:00,3.379
2015-01-02 02:00:00+01:00,3.164
2015-01-02 03:00:00+01:00,3.164
2015-01-02 04:00:00+01:00,3.164
...,...
2017-12-31 19:00:00+01:00,16.790
2017-12-31 20:00:00+01:00,16.180
2017-12-31 21:00:00+01:00,15.790


In [120]:
train_df = train_raw_df.copy()
test_df = test_raw_df.copy()

*   hour: 時 (0-23)
*  dayofweek: 曜日 (月曜日=0, 火曜日=1, ..., 日曜日=6)
*  dayofyear: 年の日 (1-366)
*  month: 月 (1-12)

In [121]:
def create_time_features(df):
    """
    DataFrameのDatetimeIndexから時間に関する特徴量を作成する関数。
    """
    # 元のDataFrameを変更しないようにコピーを作成
    df_processed = df.copy()

    # インデックスをdatetime型に変換する際に、まずUTCとして解釈させる
    # ★★★ ここを修正 ★★★
    df_processed.index = pd.to_datetime(df_processed.index, errors='coerce', utc=True)

    # UTCに設定されたインデックスを、目的のタイムゾーンに変換する
    # この行は正しく動作するようになります
    df_processed.index = df_processed.index.tz_convert("Etc/GMT-1")

    # 時間に関する特徴量を追加
    df_processed['hour'] = df_processed.index.hour
    df_processed['dayofweek'] = df_processed.index.dayofweek
    df_processed['dayofyear'] = df_processed.index.dayofyear
    df_processed['month'] = df_processed.index.month

    return df_processed

# --- 関数を使って train_df と test_df を一度に処理 ---
try:
    train_df = create_time_features(train_df)
    test_df = create_time_features(test_df)
    print("時間特徴量の作成が完了しました。")
    print(train_df[['hour', 'dayofweek', 'month']].head())
except Exception as e:
    print(f"エラーが発生しました: {e}")


時間特徴量の作成が完了しました。
                           hour  dayofweek  month
time                                             
2015-01-02 00:00:00+01:00     0          4      1
2015-01-02 01:00:00+01:00     1          4      1
2015-01-02 02:00:00+01:00     2          4      1
2015-01-02 03:00:00+01:00     3          4      1
2015-01-02 04:00:00+01:00     4          4      1


In [122]:
train_df.head()

,generation_biomass,generation_fossil_brown_coal/lignite,generation_fossil_gas,generation_fossil_hard_coal,generation_fossil_oil,generation_hydro_pumped_storage_consumption,generation_hydro_run_of_river_and_poundage,generation_hydro_water_reservoir,generation_nuclear,generation_other,...,seville_clouds_all,seville_weather_id,seville_weather_main,seville_weather_description,seville_weather_icon,price_actual,hour,dayofweek,dayofyear,month
time,,,,,,,,,,,,,,,,,,,,,
2015-01-02 00:00:00+01:00,447.0,329.0,4844.0,4821.0,162.0,863.0,1051.0,1899.0,7096.0,43.0,...,0,800,clear,sky is clear,01n,64.02,0,4,2,1
2015-01-02 01:00:00+01:00,449.0,328.0,5196.0,4755.0,158.0,920.0,1009.0,1658.0,7096.0,43.0,...,0,800,clear,sky is clear,01n,58.46,1,4,2,1
2015-01-02 02:00:00+01:00,448.0,323.0,4857.0,4581.0,157.0,1164.0,973.0,1371.0,7099.0,43.0,...,0,800,clear,sky is clear,01n,54.70,2,4,2,1
2015-01-02 03:00:00+01:00,438.0,254.0,4314.0,4131.0,160.0,1503.0,949.0,779.0,7098.0,43.0,...,0,800,clear,sky is clear,01n,54.91,3,4,2,1
2015-01-02 04:00:00+01:00,428.0,187.0,4130.0,3840.0,156.0,1826.0,953.0,720.0,7097.0,43.0,...,0,800,clear,sky is clear,01n,53.07,4,4,2,1


In [123]:
# train_dfからどの特徴量を使用するかを選択

base_features = [
    "total_load_actual",
    "generation_fossil_gas",
    "generation_fossil_hard_coal",
    "generation_fossil_oil",
    "generation_solar",
    "generation_wind_onshore",
    "generation_hydro_run_of_river_and_poundage",
    "hour",
    "dayofweek",
    "dayofyear",
    "month",
    "madrid_temp",
    "valencia_temp",
    "bilbao_temp",
    "barcelona_temp",
    "seville_temp",
    "price_actual"      # ← 目的変数
]

train_sub = train_df[base_features].copy()
test_sub = test_df[base_features[:-1]].copy() #testデータでは目的変数以外の特徴量を選択

In [124]:
# # temp_devを作成

# COMFORT = 20.0 # 快適温度 ≈ 20 ℃ と仮定
# train_sub["temp_C"] = train_sub["valencia_temp"] - 273.15 # ケルビンから摂氏に変換
# train_sub["temp_dev"] = (train_sub["temp_C"] - COMFORT).abs() # 20℃からどれぐらい離れているかを計算しtemp_devという特徴量を作成

# # testデータにも同様の特徴量を追加
# test_sub["temp_C"] = test_sub["valencia_temp"] - 273.15
# test_sub["temp_dev"] = (test_sub["temp_C"] - COMFORT).abs()

In [126]:
# 最終的にモデルで使用する特徴量をリストアップ
select_col = [
    'total_load_actual',
    'generation_solar',
    'generation_wind_onshore',
    'generation_hydro_run_of_river_and_poundage',
    'generation_fossil_gas',
    'generation_fossil_hard_coal',
    'generation_fossil_oil',
    # 'temp_dev',
    "hour",
    "dayofweek",
    "dayofyear",
    "month",
    "madrid_temp",
    "valencia_temp",
    "bilbao_temp",
    "barcelona_temp",
    "seville_temp",
    'price_actual',
]
train_selected = train_sub[select_col]
test_selected = test_sub[select_col[:-1]]

In [127]:
# 欠損値補完
train_selected.ffill(inplace=True)
test_selected.ffill(inplace=True)

In [128]:
# 目的変数以外のカラムを標準化
not_scaled_columns = ["price_actual","hour","dayofweek","dayofyear","month",]
scale_cols = [c for c in train_selected.columns if c not in not_scaled_columns]
# OKのデータフレームで作成
train_ok = train_selected.copy()

In [129]:
# OK 版：行 t 以前だけで平均・標準偏差を更新しながら標準化（expanding().mean()/std() を使う）
for col in scale_cols:
    # 平均と標準偏差を逐次更新
    train_ok[f"{col}_mean_to_t"] = train_ok[col].expanding().mean()
    train_ok[f"{col}_std_to_t"]  = train_ok[col].expanding().std(ddof=0)

    # 0除算を避けるため std==0 の行は NaN に
    train_ok[f"{col}_scaled"] = (
        (train_ok[col] - train_ok[f"{col}_mean_to_t"]) /
        train_ok[f"{col}_std_to_t"]
    )



In [130]:
print("◆ OK（リークなし）")
display(train_ok[[*(f"{c}_scaled" for c in scale_cols) ]].head())

◆ OK（リークなし）


,total_load_actual_scaled,generation_solar_scaled,generation_wind_onshore_scaled,generation_hydro_run_of_river_and_poundage_scaled,generation_fossil_gas_scaled,generation_fossil_hard_coal_scaled,generation_fossil_oil_scaled,madrid_temp_scaled,valencia_temp_scaled,bilbao_temp_scaled,barcelona_temp_scaled,seville_temp_scaled
time,,,,,,,,,,,,
2015-01-02 00:00:00+01:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-02 01:00:00+01:00,-1.000000,1.000000,-1.000000,-1.000000,1.000000,-1.000000,-1.000000,NaN,NaN,-1.000000,NaN,NaN
2015-01-02 02:00:00+01:00,-1.311200,0.707107,-1.197652,-1.192166,-0.666844,-1.363200,-0.925820,-1.414214,-1.414214,-0.411765,-1.414214,-1.414214
2015-01-02 03:00:00+01:00,-1.379864,0.577350,-1.156156,-1.207589,-1.549117,-1.637686,0.390567,-1.000000,-1.000000,-1.234083,-1.000000,-1.000000
2015-01-02 04:00:00+01:00,-1.345247,-1.984572,-1.275557,-0.885225,-1.380235,-1.544575,-1.207020,-0.816497,-0.816497,-0.927791,-0.816497,-0.816497


In [131]:
test_ok = test_selected.copy()
for col in scale_cols:
    mu_train = train_ok[f"{col}_mean_to_t"].iloc[-1]
    std_train = train_ok[f"{col}_std_to_t"].iloc[-1]
    test_ok[f"{col}_scaled"] = (test_ok[col] - mu_train) / std_train

In [132]:
def create_city_temp_deviation_features(df):
    """
    DataFrameに5都市それぞれの気温偏差特徴量を追加する関数。
    """
    # 元のDataFrameを変更しないようにコピーを作成
    df_processed = df.copy()

    # 都市名のリスト
    cities = ['valencia', 'madrid', 'bilbao', 'barcelona', 'seville']

    # 快適温度を定義（この値は調整可能です）
    COMFORT_TEMP = 15.0

    print(f"各都市の「快適温度({COMFORT_TEMP}℃)からの差」を特徴量として作成します...")

    # 各都市についてループ処理
    for city in cities:
        # 関連するカラム名を定義
        temp_k_col = f'{city}_temp'          # 元のケルビン温度カラム
        temp_c_col = f'{city}_temp_c'        # 新しく作成する摂氏温度カラム
        deviation_col = f'{city}_temp_deviation' # 最終的に作成する偏差特徴量カラム

        # 1. ケルビンから摂氏(℃)に変換
        df_processed[temp_c_col] = df_processed[temp_k_col] - 273.15

        # 2. 快適温度との差の絶対値を計算
        df_processed[deviation_col] = (df_processed[temp_c_col] - COMFORT_TEMP).abs()

    print("特徴量の作成が完了しました。")
    return df_processed

# --- 関数を使って train_ok と test_ok の両方に特徴量を追加 ---
train_ok = create_city_temp_deviation_features(train_ok)
test_ok = create_city_temp_deviation_features(test_ok)


# --- 結果の確認 ---
# 作成された新しい特徴量のカラム名リスト
new_cols = [f'{city}_temp_deviation' for city in ['valencia', 'madrid', 'bilbao', 'barcelona', 'seville']]

print("\n--- 作成された新しい特徴量の確認 (train_df_featured) ---")
print(train_ok[new_cols].head())

# print("\n--- 作成された新しい特徴量の確認 (test_df_featured) ---")
# print(test_df_featured[new_cols].head())

各都市の「快適温度(15.0℃)からの差」を特徴量として作成します...
特徴量の作成が完了しました。
各都市の「快適温度(15.0℃)からの差」を特徴量として作成します...
特徴量の作成が完了しました。

--- 作成された新しい特徴量の確認 (train_df_featured) ---
                           valencia_temp_deviation  madrid_temp_deviation  \
time                                                                        
2015-01-02 00:00:00+01:00                   16.921                 21.021   
2015-01-02 01:00:00+01:00                   16.921                 21.021   
2015-01-02 02:00:00+01:00                   17.486                 21.761   
2015-01-02 03:00:00+01:00                   17.486                 21.761   
2015-01-02 04:00:00+01:00                   17.486                 21.761   

                           bilbao_temp_deviation  barcelona_temp_deviation  \
time                                                                         
2015-01-02 00:00:00+01:00              14.675344                     5.371   
2015-01-02 01:00:00+01:00              15.647500                     5.371   


In [133]:
display(train_ok.head())

,total_load_actual,generation_solar,generation_wind_onshore,generation_hydro_run_of_river_and_poundage,generation_fossil_gas,generation_fossil_hard_coal,generation_fossil_oil,hour,dayofweek,dayofyear,...,valencia_temp_c,valencia_temp_deviation,madrid_temp_c,madrid_temp_deviation,bilbao_temp_c,bilbao_temp_deviation,barcelona_temp_c,barcelona_temp_deviation,seville_temp_c,seville_temp_deviation
time,,,,,,,,,,,,,,,,,,,,,
2015-01-02 00:00:00+01:00,25385.0,49.0,6378.0,1051.0,4844.0,4821.0,162.0,0,4,2,...,-1.921,16.921,-6.021,21.021,0.324656,14.675344,9.629,5.371,3.379,11.621
2015-01-02 01:00:00+01:00,24382.0,50.0,5890.0,1009.0,5196.0,4755.0,158.0,1,4,2,...,-1.921,16.921,-6.021,21.021,-0.647500,15.647500,9.629,5.371,3.379,11.621
2015-01-02 02:00:00+01:00,22734.0,50.0,5461.0,973.0,4857.0,4581.0,157.0,2,4,2,...,-2.486,17.486,-6.761,21.761,-0.417656,15.417656,8.714,6.286,3.164,11.836
2015-01-02 03:00:00+01:00,21286.0,50.0,5238.0,949.0,4314.0,4131.0,160.0,3,4,2,...,-2.486,17.486,-6.761,21.761,-1.089344,16.089344,8.714,6.286,3.164,11.836
2015-01-02 04:00:00+01:00,20264.0,42.0,4935.0,953.0,4130.0,3840.0,156.0,4,4,2,...,-2.486,17.486,-6.761,21.761,-1.057000,16.057000,8.714,6.286,3.164,11.836


## モデリング


1. 学習用データセットを学習用 (2017年以前) と検証用 (2017年) のデータセットへ分割
2. 学習用データセットに含まれる特徴量と目的変数の分離

In [134]:
# yearで分割するためindexをDatetimeIndex化
train_ok.index = pd.to_datetime(train_ok.index, errors="coerce", utc=True)
train_ok.index = train_ok.index.tz_convert("Etc/GMT-1")

# 2017 年をバリデーションに分割
val_mask = train_ok.index.year == 2017
train_mask = ~val_mask

In [139]:
train_ok.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 26280 entries, 2015-01-02 00:00:00+01:00 to 2017-12-31 23:00:00+01:00
Data columns (total 63 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   total_load_actual                                     26280 non-null  float64
 1   generation_solar                                      26280 non-null  float64
 2   generation_wind_onshore                               26280 non-null  float64
 3   generation_hydro_run_of_river_and_poundage            26280 non-null  float64
 4   generation_fossil_gas                                 26280 non-null  float64
 5   generation_fossil_hard_coal                           26280 non-null  float64
 6   generation_fossil_oil                                 26280 non-null  float64
 7   hour                                                  26280 non-null  int32  
 8   dayofweek

In [140]:
# 標準化した特徴量のみを選択
scaled_cols = [c for c in train_ok.columns if c.endswith("_scaled") and c.startswith("generation")]
unscaled_cols =  ["hour", "dayofweek", "dayofyear", "month", "bilbao_temp_deviation","barcelona_temp_deviation","valencia_temp_deviation", "madrid_temp_deviation", "seville_temp_deviation"]
feature_cols = scaled_cols + unscaled_cols

X_train = train_ok.loc[train_mask, feature_cols]
y_train = train_ok.loc[train_mask, 'price_actual']
X_val = train_ok.loc[val_mask, feature_cols]
y_val = train_ok.loc[val_mask, 'price_actual']
X_test = test_ok[feature_cols]

print(f"train rows: {len(X_train)}   val rows: {len(X_val)}")

train rows: 17520   val rows: 8760


In [141]:
# 「未来データの混入(=リーク)」のない標準化による1行目の欠損は0で埋める
X_train.fillna(0, inplace=True)

In [143]:
feature_cols

['generation_solar_scaled',
 'generation_wind_onshore_scaled',
 'generation_hydro_run_of_river_and_poundage_scaled',
 'generation_fossil_gas_scaled',
 'generation_fossil_hard_coal_scaled',
 'generation_fossil_oil_scaled',
 'hour',
 'dayofweek',
 'dayofyear',
 'month',
 'bilbao_temp_deviation',
 'barcelona_temp_deviation',
 'valencia_temp_deviation',
 'madrid_temp_deviation',
 'seville_temp_deviation']

In [148]:
categorical_cols = ["hour", "dayofweek", "dayofyear", "month"]

# 1. LightGBMモデルの初期化
model_lgbm = lgb.LGBMRegressor(
    objective='regression_l1',  # 損失関数 L1 (MAE) or 'regression' (L2, MSE) or 'rmse'
    metric='rmse',              # 評価指標 RMSE
    random_state=42,            # 再現性のための乱数シード
    n_estimators=100,           # 木の数 (デフォルトは100)
    # learning_rate=0.1,        # 学習率 (デフォルトは0.1)
    # num_leaves=31,            # 1本の木が持つ最大の葉の数 (デフォルトは31)
    # verbose=-1,               # 学習中のログ出力のレベル (-1で非表示)
    # 他にも多くのパラメータがあります
)

# 2. モデルの学習
print("Training LightGBM model...")
model_lgbm.fit(
    X_train,
    y_train,
    # categorical_feature=categorical_cols #カテゴリ
)
print("LightGBM model trained.")

Training LightGBM model...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000835 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3104
[LightGBM] [Info] Number of data points in the train set: 17520, number of used features: 15
[LightGBM] [Info] Start training from score 54.320000
LightGBM model trained.


In [149]:
y_pred = model_lgbm.predict(X_val)
rmse = root_mean_squared_error(y_val, y_pred)
print("\n=== 2017 Validation Metrics ===")
print(f"RMSE: {rmse:,.3f}")


=== 2017 Validation Metrics ===
RMSE: 10.935


## RMSE記録用


*   最初のベースライン（LightGBM）: 11.277
*   6/7（dayofweekなど追加）: 10.694
*   6/7（気温の偏差を追加）：10.935



## 予測・提出

In [150]:
y_test_pred = model_lgbm.predict(X_test)

In [151]:
# sample_submission に書き込み & 保存
sample_submission_df[1] = y_test_pred

In [152]:
sample_submission_df

,0,1
0,2018-01-01 00:00:00+01:00,39.536127
1,2018-01-01 01:00:00+01:00,38.011805
2,2018-01-01 02:00:00+01:00,36.299632
3,2018-01-01 03:00:00+01:00,36.095522
4,2018-01-01 04:00:00+01:00,35.442090
...,...,...
8755,2018-12-31 19:00:00+01:00,75.563365
8756,2018-12-31 20:00:00+01:00,76.201568
8757,2018-12-31 21:00:00+01:00,75.386401
8758,2018-12-31 22:00:00+01:00,69.826719


In [156]:
sample_submission_df.to_csv(submission_path+'light_gbm_0607.csv', header=False, index=False)